In [1]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))


# notebooks/02_model_training.ipynb

import pandas as pd
import numpy as np
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import plotly.graph_objects as go

# Prepare data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

# Train separate models for each household
household_models = {}
household_metrics = {}

for household in df['HouseholdID'].unique():
    print(f"\nTraining model for {household}")
    
    # Filter data for current household
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Split data
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # Train model
    predictor = ExpenditurePredictor()
    history = predictor.train(X_train, y_train)
    
    # Evaluate model
    y_pred = predictor.predict(X_test)
    mse = np.mean((y_test - y_pred.flatten()) ** 2)
    
    # Store model and metrics
    household_models[household] = predictor
    household_metrics[household] = {
        'mse': mse,
        'history': history
    }
    
    print(f"MSE for {household}: {mse:.4f}")


# Save models for each household
for household in household_models:
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household].save(model_path)
    print(f"Saved model for {household} at {model_path}")

# Visualize training history for each household
fig = go.Figure()
for household, metrics in household_metrics.items():
    history = metrics['history']
    fig.add_trace(go.Scatter(
        y=history.history['loss'],
        name=f'{household} Training Loss',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=history.history['val_loss'],
        name=f'{household} Validation Loss',
        mode='lines',
        line=dict(dash='dash')
    ))

fig.update_layout(
    title='Training History by Household',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    hovermode='x unified'
)
fig.show()

# Compare predictions vs actual for each household
for household in household_models:
    household_df = df[df['HouseholdID'] == household].copy()
    X, y = processor.prepare_training_data(household_df)
    
    # Get predictions
    y_pred = household_models[household].predict(X)
    
    # Plot actual vs predicted
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y,
        name='Actual',
        mode='lines'
    ))
    fig.add_trace(go.Scatter(
        y=y_pred.flatten(),
        name='Predicted',
        mode='lines'
    ))
    
    fig.update_layout(
        title=f'Actual vs Predicted Expenditure - {household}',
        xaxis_title='Time',
        yaxis_title='Expenditure ($)',
        hovermode='x unified'
    )
    fig.show()



Training model for H1
Epoch 1/100


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 231ms/step - loss: 0.3000 - val_loss: 0.1757
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.2698 - val_loss: 0.1501
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.2312 - val_loss: 0.1153
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1932 - val_loss: 0.0791
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.1382 - val_loss: 0.0457
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0924 - val_loss: 0.0195
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0462 - val_loss: 0.0078
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0182 - val_loss: 0.0202
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0125 - val_loss: 0.0503
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0285 - val_loss: 0.0598
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0285 - val_loss: 0.0394
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0178 - val_loss: 0.0200


c:\Users\utkar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 261ms/step - loss: 0.5099 - val_loss: 0.2639
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.4140 - val_loss: 0.2135
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.3399 - val_loss: 0.1643
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.2703 - val_loss: 0.1136
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.1925 - val_loss: 0.0659
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1198 - val_loss: 0.0297
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0468 - val_loss: 0.0215
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0182 - val_loss: 0.0510
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0327 - val_loss: 0.0735
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0437 - val_loss: 0.0577
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0267 - val_loss: 0.0344
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0173 - val_loss: 0.0225


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step 


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step 
